In [1]:
import re

def parse_cigar(cigar):
    return [(int(length), op) for length, op in re.findall(r'(\d+)(\D)', cigar)]

def build_cigar(ops):
    """Convert list of CIGAR tuples to CIGAR string"""
    return ''.join(f"{length}{op}" for op, length in ops)

def explode_cigar(ops):
    """
    Explode CIGAR operations into single-base pair elements.
    
    Args:
        ops: List of CIGAR tuples in format [(length, operation)]
        
    Returns:
        List of single-base pair operations
        
    Example:
        >>> explode_cigar([(3, 'M'), (2, 'I')])
        ['M','M','M','I', 'I']
    """
    exploded = []
    for length, op in ops:
        exploded.extend([op] * length)
    return exploded

In [3]:
from itertools import combinations

# extract positions of reference introns for the aln_ops
def exons_to_introns(exons):
    ref_intron_vec = []
    if len(exons) > 1:
        pos = 0
        for exon in exons:
            pos += (exon[1] - exon[0])
            ref_intron_vec.append(pos)
        ref_intron_vec.pop() # remove last position since it's not an intron
    return ref_intron_vec

def extract_cigar_data(cigar, qry_intron_positions):
    # Separate cigar into blocks of N and everything else
    aln_ops = []  # Stores alignment operations
    intron_blocks = []  # Stores intronic blocks

    qry_pos = 0  # Tracks the position in the query sequence - query position will never change since not affected by introns
    trg_pos = 0  # Tracks the position in the reference sequence - will depend on the intron configuration

    # Temporary storage for the current block
    current_block = []
    is_intron = False
    
    intronic_positions = dict() # positions in the original cigar that are followed by introns

    for op in cigar:
        if op == 'N':
            # If we encounter 'N', it's an intron block
            if not is_intron and current_block:
                # If we were in an alignment block, store it
                aln_ops.extend(current_block)
                current_block = []
            if not is_intron:
                ipos = len(aln_ops)
                intronic_positions.setdefault(ipos, 0) # set to 0 to indicate that this is a intorn on target, but if already set to 1, then it's a query intron and stays that way
            is_intron = True
            current_block.append('N')
            trg_pos += 1  # 'N' advances the reference position but not the query position
        else:
            if qry_pos in qry_intron_positions:
                ipos = len(aln_ops)
                if not is_intron:
                    ipos += len(current_block)
                intronic_positions[ipos] = 1 # set to 1 to indicate that this is a query position
            # If we encounter anything else, it's an alignment operation
            if is_intron and current_block:
                # If we were in an intron block, store it
                intron_blocks.append(current_block)
                current_block = []
            is_intron = False
            current_block.append((op, qry_pos, trg_pos))
            if op in ['M', 'X', '=']:  # These operations advance both query and reference positions
                qry_pos += 1
                trg_pos += 1
            elif op == 'S': # Soft clipping advances query position only
                qry_pos += 1
            elif op == 'I':  # Insertion advances query position only
                qry_pos += 1
            elif op == 'D':  # Deletion advances reference position only
                trg_pos += 1

    # Append the last block if it exists
    if current_block:
        if is_intron:
            intron_blocks.append(current_block)
        else:
            aln_ops.extend(current_block)
            
    return aln_ops, intron_blocks, intronic_positions

def score_cigar(cigar, qry_intron_match_score, trg_pos_match_score, trg_pos_mismatch_score):
    score = 0

    qry_pos = 0  # Tracks the position in the query sequence - query position will never change since not affected by introns
    trg_pos = 0  # Tracks the position in the reference sequence - will depend on the intron configuration

    is_intron = False

    for op in cigar:
        if op[0] == 'N':
            if not is_intron: # first intronic position - check the score
                score += op[1] * qry_intron_match_score # op[1] set to 1 if matches the query intron position, 0 otherwise
            is_intron = True
            trg_pos += 1  # 'N' advances the reference position but not the query position
        else:
            is_intron = False
            if trg_pos == op[2]: # matching position
                score += trg_pos_match_score
            else:
                score += trg_pos_mismatch_score
            if op[0] in ['M', 'X', '=']:  # These operations advance both query and reference positions
                qry_pos += 1
                trg_pos += 1
            elif op[0] == 'S': # Soft clipping advances query position only
                qry_pos += 1
            elif op[0] == 'I':  # Insertion advances query position only
                qry_pos += 1
            elif op[0] == 'D':  # Deletion advances reference position only
                trg_pos += 1
                
    return score

def merge_ops(ops):
    merged_cigar = []
    count = 1
    for i in range(1, len(ops)):
        if ops[i][0] == ops[i-1][0]:
            count += 1
        else:
            merged_cigar.append((ops[i-1][0], count))
            count = 1
    merged_cigar.append((ops[-1][0], count))
    return merged_cigar


In [4]:
def generate_cigar_variations(ops, intronic_positions, intron_blocks):
    """
    Generate all possible cigar variations by inserting introns at specified positions.

    Args:
        ops (list): The input list of operations.
        intronic_positions (list): List of positions in the list where insertion can happen.
        intron_blocks (list): List of strings to insert in the order specified.

    Returns:
        list: A list of all possible variations of the list `s`.
    """
    if len(intron_blocks) > len(intronic_positions):
        raise ValueError("Number of intron_blocks cannot be greater than the number of intronic positions.")

    cigars = []

    # Generate all combinations of positions to insert the strings
    for pos_comb in combinations(intronic_positions, len(intron_blocks)):
        temp_s = ops[:]
        offset = 0  # To adjust the insertion index as the list changes
        for iblock, pos in zip(intron_blocks, pos_comb):
            temp_s = temp_s[:pos + offset] + [(x,intronic_positions[pos]) for x in iblock] + temp_s[pos + offset:]
            offset += len(iblock)  # Update offset after insertion
        cigars.append(temp_s)

    return cigars

In [14]:
original_cigar = '5M3N4M'
raw_ops = parse_cigar(original_cigar)
ops = explode_cigar(raw_ops)

exons = [(1, 6), (10, 14)]
qry_intron_positions = exons_to_introns(exons)

aln_ops, intron_blocks, trg_intronic_positions = extract_cigar_data(ops, qry_intron_positions)

# generate all possible variations of the alignment operations
cigar_variants = generate_cigar_variations(aln_ops, trg_intronic_positions, intron_blocks)

# Print the results
print("Target Intronic Positions:", trg_intronic_positions)

# now we need to score each variant
qry_intron_match_score = 10
trg_pos_match_score = 1
trg_pos_mismatch_score = -1

for cvar in cigar_variants:
    score = score_cigar(cvar, qry_intron_match_score, trg_pos_match_score, trg_pos_mismatch_score)
    print(build_cigar(merge_ops(cvar)), score)

Target Intronic Positions: {5: 1}
5M3N4M 19


In [18]:
original_cigar = '288M5238N102M3I154M8D3M1D16M12I381M45N22M3I111M1D1M1D1M1D366M5I1M1I29M2D3M1D92M1D4M4I121M3I2M1I6M2I20M3I7M3I144M6D6M3D6M1I1M2I1866M1I151M'
raw_ops = parse_cigar(original_cigar)
ops = explode_cigar(raw_ops)

exons = [(454, 743), (5976, 9635)]
qry_intron_positions = exons_to_introns(exons)

aln_ops, intron_blocks, trg_intronic_positions = extract_cigar_data(ops, qry_intron_positions)

# generate all possible variations of the alignment operations
cigar_variants = generate_cigar_variations(aln_ops, trg_intronic_positions, intron_blocks)

# Print the results
print("Target Intronic Positions:", trg_intronic_positions)

# now we need to score each variant
qry_intron_match_score = 10
trg_pos_match_score = 1
trg_pos_mismatch_score = -1

for cvar in cigar_variants:
    score = score_cigar(cvar, qry_intron_match_score, trg_pos_match_score, trg_pos_mismatch_score)
    print(build_cigar(merge_ops(cvar)), score)

Target Intronic Positions: {288: 0, 290: 1, 968: 0}
288M5238N2M45N100M3I154M8D3M1D16M12I403M3I111M1D1M1D1M1D366M5I1M1I29M2D3M1D92M1D4M4I121M3I2M1I6M2I20M3I7M3I144M6D6M3D6M1I1M2I1866M1I151M 2627
288M5238N102M3I154M8D3M1D16M12I381M45N22M3I111M1D1M1D1M1D366M5I1M1I29M2D3M1D92M1D4M4I121M3I2M1I6M2I20M3I7M3I144M6D6M3D6M1I1M2I1866M1I151M 3973
290M5238N100M3I154M8D3M1D16M12I381M45N22M3I111M1D1M1D1M1D366M5I1M1I29M2D3M1D92M1D4M4I121M3I2M1I6M2I20M3I7M3I144M6D6M3D6M1I1M2I1866M1I151M 3979


In [ ]:

exons = [(11, 13), (15, 18)]
qry_intron_positions = exons_to_introns(exons)
print("Query Intron Positions on Query:", qry_intron_positions)

ops = list("MMNNMMMMNM")

aln_ops, intron_blocks, trg_intronic_positions = extract_cigar_data(ops, qry_intron_positions)

# generate all possible variations of the alignment operations
cigar_variants = generate_cigar_variations(aln_ops, trg_intronic_positions, intron_blocks)

# Print the results
print("Alignment Operations:", aln_ops)
print("Intronic Blocks:", intron_blocks)
print("Target Intronic Positions:", trg_intronic_positions)
print("Variants:")
for variant in cigar_variants:
    print(cigar_variants)

# now we need to score each variant
qry_intron_match_score = 10
trg_pos_match_score = 1
trg_pos_mismatch_score = -1

for cvar in cigar_variants:
    score = score_cigar(cvar, qry_intron_match_score, trg_pos_match_score, trg_pos_mismatch_score)
    print(cvar, score)
    
# pick variant with highest score
# if multiple variants have the same score, pick the first
best_cigar = max(cigar_variants, key=lambda x: score_cigar(x, qry_intron_match_score, trg_pos_match_score, trg_pos_mismatch_score))
best_cigar

merged_cigar = merge_ops(best_cigar)
merged_cigar